#### WPE & WAPE Calculation

In [1]:
!pip install uritools

In [2]:
import pandas as pd
import numpy as np
import boto3
import pyarrow.parquet as pq
import s3fs
import math
import yaml

import sys
sys.path.append('../../')
from src.utils import read_multipart_parquet_s3, write_df_to_parquet_on_s3,read_csv_s3

In [3]:
with open('monitor_whitelist.yaml') as config_file:
    monitor_wl = yaml.full_load(config_file)

##### Fetch data

In [13]:
df_tree = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_tree', prefix_filename='part-')
df_tree = df_tree[(df_tree.week_id == 202211)]

In [68]:
df_tree[(df_tree.model_id == 8301690) & (df_tree.week_id >= 202206)]

,model_id,week_id,family_id,sub_department_id,department_id,univers_id,product_nature_id,model_label,family_label,sub_department_label,department_label,univers_label,product_nature_label,brand_label,brand_type
2682312,8301690,202211,7026,1618,400,4,25179,ONRHYTHM 110 BLACK,WATCHES HRM PED,ELECTRONIC,RUNNING TRAIL A,RUNNING WALKING,HEART RATE MONITOR,KALENJI,MP


In [70]:
fcst_bi_static_feat = read_csv_s3('fcst-workspace', 
                                   'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_static_feat_w.csv')

In [71]:
fcst_bi_static_feat = fcst_bi_static_feat.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_static_feat = fcst_bi_static_feat[(fcst_bi_static_feat.family_id.isin(monitor_wl['pre_selection_family']))]

In [73]:
fcst_bi_static_feat[['model_id', 'wpe_reach_target', 'wape_reach_target',
       'univers_label', 'department_label', 'family_label', 'rs_email',
       'AW2021_CodeA', 'tier', 'model_label', 'ec_tag', 'selling_length',
       'aw_seasonality', 'annual_seasonality', 'wpe_cutoff_ok_pct',
       'wape_cutoff_ok_pct', 'update_time']].to_csv('fcst_bi_static_feat_w.csv',index=False)

In [74]:
fcst_bi_dynamic_feat = read_csv_s3('fcst-workspace', 
                                   'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_dynamic_feat.csv')

In [75]:
fcst_bi_dynamic_feat = fcst_bi_dynamic_feat.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_dynamic_feat = fcst_bi_dynamic_feat[(fcst_bi_dynamic_feat.family_id.isin(monitor_wl['pre_selection_family']))]

In [76]:
fcst_bi_dynamic_feat[['model_id','week_id','weekly_average_price','num_store_following']]\
.to_csv('fcst_bi_dynamic_feat.csv',index=False)

In [10]:
fcst_bi_forecast_result = read_csv_s3('fcst-workspace', 
                                      'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_forecast_result.csv')

In [11]:
fcst_bi_forecast_result[(fcst_bi_forecast_result.algorithm == 'realized_sales') \
                        & (fcst_bi_forecast_result.qty_tag == 'sales_w')].quantity.sum()

#51537265/152819899

155040168.0

In [14]:
fcst_bi_forecast_result = fcst_bi_forecast_result.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_forecast_result = fcst_bi_forecast_result[(fcst_bi_forecast_result.family_id.isin(monitor_wl['pre_selection_family']))]

In [15]:
df_sales = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                     'global/model_week_sales', prefix_filename='part-')

In [16]:
df_sales = df_sales.groupby(by=['model_id','week_id'],as_index=False).agg({'sales_quantity':'sum'})

In [24]:
fcst_bi_forecast_result_1 = fcst_bi_forecast_result[(fcst_bi_forecast_result.algorithm != 'realized_sales')][['model_id','week_id','algorithm','qty_tag','quantity']]
model = fcst_bi_forecast_result_1.model_id.unique()

In [25]:
#sales_w
df = df_sales[(df_sales.model_id.isin(model)) & (df_sales.week_id >= 202101) & (df_sales.week_id <= 202210)]
df['algorithm'] = 'realized_sales'
df['qty_tag'] = 'sales_w'
df = df.rename(columns = {'sales_quantity':'quantity'})
df = df[['model_id','week_id','algorithm','qty_tag','quantity']]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
df2 = df_sales[(df_sales.model_id.isin(model)) & (df_sales.week_id >= 202001) & (df_sales.week_id <= 202110)]
df2['week_id'] = df2['week_id'].apply(lambda x:x+100)
df2['algorithm'] = 'realized_sales'
df2['qty_tag'] = 'sales_52w_bf'
df2 = df2.rename(columns = {'sales_quantity':'quantity'})
df2 = df2[['model_id','week_id','algorithm','qty_tag','quantity']]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [36]:
df3 = df_sales[(df_sales.model_id.isin(model)) & (df_sales.week_id >= 201901) & (df_sales.week_id <= 202010)]
df3['week_id'] = df3['week_id'].apply(lambda x:x+200)
df3['algorithm'] = 'realized_sales'
df3['qty_tag'] = 'sales_104w_bf'
df3 = df3.rename(columns = {'sales_quantity':'quantity'})
df3 = df3[['model_id','week_id','algorithm','qty_tag','quantity']]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [42]:
fcst_bi_forecast_result = fcst_bi_forecast_result_1.append(df).append(df2).append(df3)

In [43]:
fcst_bi_forecast_result[['model_id','week_id','algorithm','qty_tag','quantity']]\
.to_csv('fcst_bi_forecast_result.csv',index=False)

In [82]:
fcst_bi_forecast_performance = read_csv_s3('fcst-workspace', 
                                   'forecast-cn/fcst-data-exchange-dev/dashboard/tableau/total_cutoff/fcst_bi_forecast_performance_a.csv')

In [83]:
fcst_bi_forecast_performance = fcst_bi_forecast_performance.merge(df_tree[['model_id','family_id']],on='model_id')
fcst_bi_forecast_performance = fcst_bi_forecast_performance[(fcst_bi_forecast_performance.family_id.isin(monitor_wl['pre_selection_family']))]

In [87]:
fcst_bi_forecast_performance.to_csv('fcst_bi_forecast_performance_a.csv',index=False)

In [88]:
fcst_bi_forecast_performance[['model_id']].drop_duplicates().to_csv('model_id.csv',index=False)

In [89]:
fcst_bi_static_feat[['model_id']].drop_duplicates().to_csv('model_id.csv',index=False)

#### Check

In [91]:
df_sales = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_sales', prefix_filename='part-')

In [99]:
df_sales[(df_sales.model_id == 8552490) & (df_sales.week_id.isin([202106,202107,202108,202109,202110]))]

,model_id,week_id,date,channel,sales_quantity
333253,8552490,202106,2021-02-07,offline,600
333347,8552490,202107,2021-02-14,online,18
713318,8552490,202108,2021-02-21,online,15
1543057,8552490,202110,2021-03-07,online,20
2314819,8552490,202108,2021-02-21,offline,549
3181716,8552490,202107,2021-02-14,offline,800
3941196,8552490,202109,2021-02-28,online,4
5222200,8552490,202106,2021-02-07,online,9
5222517,8552490,202109,2021-02-28,offline,582
5222935,8552490,202110,2021-03-07,offline,677
